# Simulated Annealing grid search 

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from copy import deepcopy
from functools import partial
import os
from itertools import product
import json
import math


In [20]:
import sys

sys.path.append("..")

from functions.utils import *
from functions.crossover import *
from functions.selection_algos import *
from functions.mutations import *
from functions.algorithms import *
from functions.solutions import *

In [21]:
scores = pd.read_csv('data/seating_data(in).csv', index_col=0)
scores_array = scores.to_numpy()


> For simulated annealing we will run with maximum 100 iterations and 50 local search iterations, meaning that we will have 5_000 fitness evaluations as we did with the genetic algorithm (50 pop_size * 100 n_generations = 5_000).


> we will store the results in "sa_results.csv"

In [22]:
results_file = "results/sa_results.csv"

In [23]:
max_iters = [100] 
L_values = [50] 
neighbor_ops = [scramble_mutation_optimized, swap_mutation]
C_values = [1, 10, 100, 200]
H_values = [1.01, 1.1, 1.2, 1.3]
k_values = [1, 2, 3]

if os.path.exists(results_file):
    df_existing = pd.read_csv(results_file)
    tested_configs = set()
    for _, row in df_existing.iterrows():
        k_val = row["k"] if not pd.isna(row["k"]) else None
        config_key = (row["max_iter"], row["L"], row["neighbor"], k_val, row["C"], row["H"])
        tested_configs.add(config_key)
else:
    df_existing = pd.DataFrame()
    tested_configs = set()


for max_iter, L, neighbor_fn, C, H in product(max_iters, L_values, neighbor_ops, C_values, H_values):
    if neighbor_fn.__name__ == "scramble_mutation_optimized":
        k_vals = k_values 
    else:
        k_vals = [None]
    
    for k in k_vals:
        if k is not None:
            neighbor_callable = partial(scramble_mutation_optimized, k=k)
        else:
            neighbor_callable = neighbor_fn

        config_key = (max_iter, L, neighbor_fn.__name__, k, C, H)

        if config_key in tested_configs:
            print(f"Skipping tested config: {config_key}")
            continue

        print(f"Running SA with config: {config_key}")

        fitnesses, conv_iters, conv_times = [], [], []

        for _ in range(30):
            sol = Wedding_SimulatedAnnealing_Solution(scores=scores_array)

            best_sol, _, best_iter, conv_time = simulated_annealing(
                initial_solution=sol,
                C=C,
                L=L,
                H=H,
                maximization=True,
                max_iter=max_iter,
                verbose=False,
                neighbor_operator=neighbor_callable
            )

            fitnesses.append(best_sol.fitness())
            conv_iters.append(best_iter)
            conv_times.append(conv_time)

        avg_fit = round(np.mean(fitnesses), 2)
        std_fit = round(np.std(fitnesses), 2)
        avg_iter = round(np.mean(conv_iters), 2)
        avg_time = round(np.mean(conv_times), 2)

        print(f"Avg fitness: {avg_fit}, Std: {std_fit}, Iter: {avg_iter}, Time: {avg_time}s")

        df_new = pd.DataFrame([{
            "max_iter": max_iter,
            "L": L,
            "neighbor": neighbor_fn.__name__,
            "k": k,
            "C": C,
            "H": H,
            "avg_fitness": avg_fit,
            "std_fitness": std_fit,
            "avg_iter": avg_iter,
            "avg_time": avg_time,
            "fitness_scores": json.dumps([float(f) for f in fitnesses])
        }])

        if os.path.exists(results_file):
            df_new.to_csv(results_file, mode="a", index=False, header=False)
        else:
            df_new.to_csv(results_file, index=False)

        tested_configs.add(config_key)

Skipping tested config: (100, 50, 'scramble_mutation_optimized', 1, 1, 1.01)
Skipping tested config: (100, 50, 'scramble_mutation_optimized', 2, 1, 1.01)
Skipping tested config: (100, 50, 'scramble_mutation_optimized', 3, 1, 1.01)
Skipping tested config: (100, 50, 'scramble_mutation_optimized', 1, 1, 1.1)
Skipping tested config: (100, 50, 'scramble_mutation_optimized', 2, 1, 1.1)
Skipping tested config: (100, 50, 'scramble_mutation_optimized', 3, 1, 1.1)
Skipping tested config: (100, 50, 'scramble_mutation_optimized', 1, 1, 1.2)
Skipping tested config: (100, 50, 'scramble_mutation_optimized', 2, 1, 1.2)
Skipping tested config: (100, 50, 'scramble_mutation_optimized', 3, 1, 1.2)
Skipping tested config: (100, 50, 'scramble_mutation_optimized', 1, 1, 1.3)
Skipping tested config: (100, 50, 'scramble_mutation_optimized', 2, 1, 1.3)
Skipping tested config: (100, 50, 'scramble_mutation_optimized', 3, 1, 1.3)
Skipping tested config: (100, 50, 'scramble_mutation_optimized', 1, 10, 1.01)
Skippin

# Evaluate top 10 configurations

> In order to compare the results of the best configurations between the algorithms we will run the algorithm for 10_000 fitness evaluations each (100 max_iter * 100 L). <br>
> We will chose the top 10 configurations


In [24]:
df = pd.read_csv("results/sa_results.csv")


df["fitness_list"] = df["fitness_scores"].apply(json.loads)

df["fitness_median"] = df["fitness_list"].apply(np.median)

df = df.sort_values(by=["fitness_median"], ascending=False)

top_configs = df.head(10)
top_configs

,max_iter,L,neighbor,k,C,H,avg_fitness,std_fitness,avg_iter,avg_time,fitness_scores,fitness_list,fitness_median
0,100,50,scramble_mutation_optimized,3.0,100,1.01,73853.33,2118.13,89.77,0.31,"[71600.0, 74300.0, 74400.0, 73100.0, 72300.0, ...","[71600.0, 74300.0, 74400.0, 73100.0, 72300.0, ...",74250.0
1,100,50,scramble_mutation_optimized,3.0,1,1.01,73283.33,3231.42,87.80,0.31,"[71900.0, 68500.0, 68000.0, 69600.0, 72700.0, ...","[71900.0, 68500.0, 68000.0, 69600.0, 72700.0, ...",74100.0
2,100,50,scramble_mutation_optimized,3.0,100,1.10,72776.67,3152.00,90.43,0.33,"[67700.0, 70700.0, 77000.0, 71700.0, 69400.0, ...","[67700.0, 70700.0, 77000.0, 71700.0, 69400.0, ...",73600.0
3,100,50,scramble_mutation_optimized,3.0,10,1.01,73136.67,2148.41,88.73,0.31,"[74900.0, 70700.0, 75400.0, 73800.0, 75700.0, ...","[74900.0, 70700.0, 75400.0, 73800.0, 75700.0, ...",73450.0
4,100,50,scramble_mutation_optimized,3.0,200,1.01,73120.00,2318.25,91.37,0.32,"[72000.0, 76400.0, 73300.0, 77100.0, 72800.0, ...","[72000.0, 76400.0, 73300.0, 77100.0, 72800.0, ...",73300.0
5,100,50,scramble_mutation_optimized,2.0,1,1.01,72796.67,2216.38,87.97,0.30,"[72200.0, 68800.0, 74200.0, 74100.0, 78400.0, ...","[72200.0, 68800.0, 74200.0, 74100.0, 78400.0, ...",73150.0
6,100,50,scramble_mutation_optimized,3.0,10,1.30,73116.67,3001.45,88.17,0.31,"[72500.0, 77700.0, 72900.0, 70100.0, 74700.0, ...","[72500.0, 77700.0, 72900.0, 70100.0, 74700.0, ...",73150.0
7,100,50,scramble_mutation_optimized,3.0,200,1.10,73146.67,2460.18,92.10,0.33,"[72300.0, 76400.0, 73700.0, 75100.0, 74000.0, ...","[72300.0, 76400.0, 73700.0, 75100.0, 74000.0, ...",73150.0
8,100,50,scramble_mutation_optimized,3.0,10,1.20,73156.67,1514.53,88.83,0.31,"[73900.0, 74600.0, 74500.0, 70000.0, 70100.0, ...","[73900.0, 74600.0, 74500.0, 70000.0, 70100.0, ...",73100.0
9,100,50,scramble_mutation_optimized,2.0,200,1.01,71966.67,3241.12,90.00,0.30,"[74100.0, 68700.0, 75700.0, 72000.0, 67800.0, ...","[74100.0, 68700.0, 75700.0, 72000.0, 67800.0, ...",73100.0


> Aggregate the top configuration for the other non selected neighboor operator (swap mutation) 

In [25]:
top_config_swap = df[df["neighbor"] == "swap_mutation"].iloc[0]
top_config_swap = top_config_swap.to_frame().T

top_config_all = pd.concat([top_configs, top_config_swap], ignore_index=True)
top_config_all.tail(3)

/var/folders/jb/jm3xb4f53hs9mcvbg93nks7h0000gn/T/ipykernel_61322/3796238901.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  top_config_all = pd.concat([top_configs, top_config_swap], ignore_index=True)


,max_iter,L,neighbor,k,C,H,avg_fitness,std_fitness,avg_iter,avg_time,fitness_scores,fitness_list,fitness_median
8,100,50,scramble_mutation_optimized,3.0,10,1.2,73156.67,1514.53,88.83,0.31,"[73900.0, 74600.0, 74500.0, 70000.0, 70100.0, ...","[73900.0, 74600.0, 74500.0, 70000.0, 70100.0, ...",73100.0
9,100,50,scramble_mutation_optimized,2.0,200,1.01,71966.67,3241.12,90.0,0.3,"[74100.0, 68700.0, 75700.0, 72000.0, 67800.0, ...","[74100.0, 68700.0, 75700.0, 72000.0, 67800.0, ...",73100.0
10,100,50,swap_mutation,NaN,1,1.3,62920.0,2429.05,66.63,0.23,"[65400.0, 60800.0, 62700.0, 66400.0, 59700.0, ...","[65400.0, 60800.0, 62700.0, 66400.0, 59700.0, ...",62800.0


In [26]:
top_config_k1 = df[df["k"] == 1].iloc[0]
top_config_k1 = top_config_k1.to_frame().T

top_config_all = pd.concat([top_config_all, top_config_k1], ignore_index=True)
top_config_all.tail(3)

,max_iter,L,neighbor,k,C,H,avg_fitness,std_fitness,avg_iter,avg_time,fitness_scores,fitness_list,fitness_median
9,100,50,scramble_mutation_optimized,2.0,200,1.01,71966.67,3241.12,90.0,0.3,"[74100.0, 68700.0, 75700.0, 72000.0, 67800.0, ...","[74100.0, 68700.0, 75700.0, 72000.0, 67800.0, ...",73100.0
10,100,50,swap_mutation,NaN,1,1.3,62920.0,2429.05,66.63,0.23,"[65400.0, 60800.0, 62700.0, 66400.0, 59700.0, ...","[65400.0, 60800.0, 62700.0, 66400.0, 59700.0, ...",62800.0
11,100,50,scramble_mutation_optimized,1.0,200,1.01,69043.33,2317.71,91.23,0.3,"[70700.0, 69100.0, 69200.0, 74800.0, 71000.0, ...","[70700.0, 69100.0, 69200.0, 74800.0, 71000.0, ...",69100.0


In [27]:
num_runs = 100
max_gen = 100
L = 100 

results_file = "results/avg_fitness_by_generation_sa.csv"


scores = pd.read_csv('data/seating_data(in).csv', index_col=0)
scores_array = scores.to_numpy()

# Create the CSV with header if it doesn't exist
if os.path.exists(results_file):
    df_existing = pd.read_csv(results_file)
    tested_configs = set(row["configuration"] for _, row in df_existing.iterrows())
else:
    df_existing = pd.DataFrame()
    tested_configs = set()
    

for _, row in top_config_all.iterrows():
    config_key = f"(100, 100, '{row['neighbor']}', {row['k']}, {row['C']}, {row['H']})"
    
    if config_key in tested_configs:
        print(f"Configuration {config_key} already tested. Skipping...")
        continue
    
    if row["neighbor"] == "scramble_mutation_optimized":
        k = float(row["k"])
        neighbor_fn = partial(scramble_mutation_optimized, k=k)
    
    fitness_gen_all = []
    conv_iter_all = []
    best_sol_fitness = []
    fitness_hists = []
    
    print(f"Running configuration: {config_key}")
    for _ in range(num_runs):
        
        best_sol, fitness_hist, conv_iter, _ = simulated_annealing(
            Wedding_SimulatedAnnealing_Solution(scores=scores_array),
            max_iter=100,
            L=100,
            neighbor_operator=neighbor_fn,
            C=float(row["C"]),
            H=float(row["H"]),
        )
        
        fitness_gen_all.append(fitness_hist)
        conv_iter_all.append(conv_iter)
        best_sol_fitness.append(best_sol.fitness())
        
    median_fitness_by_gen = np.median(fitness_gen_all, axis=0)
    print(f"Convergence iterations: {round(np.mean(conv_iter_all),2)}")
    
    df_new = pd.DataFrame([{
        "configuration": config_key,
        "fitness_by_gen": json.dumps([float(x) for x in median_fitness_by_gen]),
        "best_fitness": json.dumps([float(x) for x in best_sol_fitness]),
    }])
    
    if os.path.exists(results_file) and not df_new.empty:
        df_new.to_csv(results_file, mode='a', header=False, index=False)
    else:
        df_new.to_csv(results_file, index=False)
    
    tested_configs.add(config_key)

Configuration (100, 100, 'scramble_mutation_optimized', 3.0, 100, 1.01) already tested. Skipping...
Configuration (100, 100, 'scramble_mutation_optimized', 3.0, 1, 1.01) already tested. Skipping...
Configuration (100, 100, 'scramble_mutation_optimized', 3.0, 100, 1.1) already tested. Skipping...
Configuration (100, 100, 'scramble_mutation_optimized', 3.0, 10, 1.01) already tested. Skipping...
Configuration (100, 100, 'scramble_mutation_optimized', 3.0, 200, 1.01) already tested. Skipping...
Configuration (100, 100, 'scramble_mutation_optimized', 2.0, 1, 1.01) already tested. Skipping...
Configuration (100, 100, 'scramble_mutation_optimized', 3.0, 10, 1.3) already tested. Skipping...
Configuration (100, 100, 'scramble_mutation_optimized', 3.0, 200, 1.1) already tested. Skipping...
Configuration (100, 100, 'scramble_mutation_optimized', 3.0, 10, 1.2) already tested. Skipping...
Configuration (100, 100, 'scramble_mutation_optimized', 2.0, 200, 1.01) already tested. Skipping...
Configurati